In [ ]:
import numpy as np
from games.tictactoe import TickTackToe
from models.mcts import MCTS
from models.resnet import ResNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt

In [ ]:
class DeepZero:
    def __init__(self, model, optimizer, game, args):
        self.model = model
        self.optimizer = optimizer
        self.game = game


In [ ]:
tictactoe = TickTackToe()
player = 1

args = {
    'C': 2,
    'num_search': 1000,
}

model = ResNet(tictactoe, 4, 64)
model.eval()

mcts = MCTS(tictactoe, args, model)

state = tictactoe.get_initial_state()

In [8]:
while True:
    print(state)
    if player == 1:
        valid_moves = tictactoe.get_valid_moves(state)
        print("val_movies", [i for i in range(tictactoe.action_size) if valid_moves[i] == 1])
        action = int(input(f"{player}: "))

        if valid_moves[action] == 0:
            print("action not val")
            continue
    else:
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)

    state = tictactoe.get_next_state(state, action, player)

    value, is_terminate = tictactoe.get_value_and_terminated(state, action)

    if is_terminate:
        if value == 1:
            print(player, "win")
        else:
             print(player, "lose")
        break

    player = tictactoe.get_opponent(player)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
val_movies [0, 1, 2, 3, 4, 5, 6, 7, 8]
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
val_movies [1, 2, 3, 5, 6, 7, 8]
[[ 1.  0.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
[[ 1. -1.  1.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
val_movies [3, 5, 6, 7, 8]
[[ 1. -1.  1.]
 [ 0. -1.  0.]
 [ 0.  1.  0.]]
[[ 1. -1.  1.]
 [ 0. -1. -1.]
 [ 0.  1.  0.]]
val_movies [3, 6, 8]
[[ 1. -1.  1.]
 [ 1. -1. -1.]
 [ 0.  1.  0.]]
[[ 1. -1.  1.]
 [ 1. -1. -1.]
 [-1.  1.  0.]]
val_movies [8]
action not val
[[ 1. -1.  1.]
 [ 1. -1. -1.]
 [-1.  1.  0.]]
val_movies [8]
1 lose


In [ ]:
while True:
    print(state)
    if player == 1:
        valid_moves = tictactoe.get_valid_moves(state)
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)

        if valid_moves[action] == 0:
            print("action not val")
            continue
    else:
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)

    state = tictactoe.get_next_state(state, action, player)

    value, is_terminate = tictactoe.get_value_and_terminated(state, action)

    if is_terminate:
        if value == 1:
            print(player, "win")
        else:
            print(player, "lose")
        break

    player = tictactoe.get_opponent(player)